In [1]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'PV_ICE' / 'TEMP' / 'ElectricFutures')

# Another option using relative address; for some operative systems you might need '/' instead of '\'
# testfolder = os.path.abspath(r'..\..\PV_DEMICE\TEMP')  

print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP\ElectricFutures


In [27]:
MATERIALS = ['glass','silver','silicon', 'copper','aluminium_frames']
MATERIAL = MATERIALS[0]

MODULEBASELINE = r'..\..\PV_ICE\baselines\LiteratureProjections\EF-CapacityByState-basecase.csv'
MODULEBASELINE_High = r'..\..\PV_ICE\baselines\LiteratureProjections\EF-CapacityByState-LowREHighElec.csv'


In [28]:
import PV_ICE
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [29]:
PV_ICE.__version__

'0+untagged.364.g911d04e.dirty'

In [30]:
plt.rcParams.update({'font.size': 22})
plt.rcParams['figure.figsize'] = (12, 5)

In [42]:
df = pd.read_csv(MODULEBASELINE)
df.set_index(['time','gid'], inplace=True)


In [43]:
#df.set_index('time', inplace=True)
#df.index=pd.PeriodIndex(df.index, freq='A')


In [47]:
df['time']

KeyError: 'time'